# Libs, Params and Data

## Libs

In [1]:
import os
import pandas as pd
pd.options.display.max_columns = 100
import numpy as np
import joblib
import seaborn as sns  
import altair as alt
from itertools import product
from tqdm.notebook import tqdm

In [2]:
from sklearn.model_selection import (train_test_split, GridSearchCV, 
                                     RepeatedStratifiedKFold, RepeatedKFold,
                                     StratifiedShuffleSplit, LeaveOneOut,
                                     cross_val_score)

In [3]:
#from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn import BalancedRandomForestClassifier

ModuleNotFoundError: No module named 'imblearn'

In [9]:
# Import local utils
from utils import (rf_feat_importance, plot_fi,
                   cluster_columns, report_AUC,
                   test_eval_repetitions,
                   plot_rocs_from_multiple_runs,
                   simple_altair_bar_chart,
                   build_partial_dep_df,
                   partial_dep_graph,
                   get_oob,extract_AUC_from_trees,
                   plot_AUC_by_num_trees,
                   waterfall_from_model_and_row,
                   plot_roc_curve_with_baseline,
                   get_oob_cv_balancedRF)

import seaborn as sns  
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'imblearn'

In [10]:
!pip install imbalanced-learn

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:719)'),)) - skipping


DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


## Utils

In [4]:
def plot_bars_percent_firstk(df, colname, title=None, k=None, other='Otros', show_text=True):
    if not title:
        title = f'Top {k} Frequency by {colname}'
    to_keep = df.groupby(colname, dropna=False).size().sort_values(ascending=False)
    if k:
        to_keep = to_keep.head(k)
    to_keep = to_keep.index.tolist()
    df_ = df.copy()
    df_[colname] = df[colname].map(dict(zip(to_keep,to_keep))).fillna(other)
    df_groupped = (df_
     .groupby(colname)
     .size()
     .reset_index(name='count'))
    return plot_bars_percent_groupped(df_groupped=df_groupped, colname=colname, 
                                      title=title, show_text=show_text, sort=True)

def plot_bars_percent_groupped(df_groupped, colname, title, color_colname=None, show_text=True, 
                                count_col='count', rot=False, legend=None, sort=False):
    # Without color_colname
    if not color_colname:
        color_colname = colname
    
    color = alt.Color(color_colname+':N', title=legend)
    y_data = alt.Y(colname+':N')
    if sort:
        y_data = alt.Y(colname+':N', sort='-x')
        
    df_to_show = (df_groupped
     .assign(percent = lambda x: 100*x[count_col]/x[count_col].sum())
     .round(2)
    )
    
    if rot:
        aux = colname
        colname = color_colname
        color_colname = aux
    
    base = alt.Chart(df_to_show).encode(
        y = y_data,
        x = count_col
    )
    
    bars = base.mark_bar().encode(
        color = color,
        tooltip = [colname, count_col, 'percent']
    ).properties(
        title = title
    )
    
    if show_text:
        text = base.mark_text(dx=14, dy=1, color='black').encode(
            text=alt.Text('percent:Q', format='.1f')
        )
        bars = bars + text

        
    return bars

def plots_bar_custom(df_, colname, score_col, title, map_scale=None, color_colname=None, show_text=False, 
                     count_col='count', rot=False, legend=None):
    # Without color_colname
    if not color_colname:
        color_colname = colname
        
    # Without map_scale
    if not map_scale:
        map_scale = {
            0: '0-99',
            100: '100-199',
            200: '200-299',
            300: '300-399',
            400: '400-499',
            500: '500-599',
            600: '600-699',
            700: '700-799',
            800: '800-899',
            900: '900-1000',
        }
        
    dt_to_show = df_
    dt_to_show[colname] = 100*(dt_to_show[score_col]/100).astype(int)
    dt_to_show = dt_to_show.copy()
    dt_to_show.loc[dt_to_show[colname]==1000, colname] = 900
    dt_to_show[colname] = dt_to_show[colname].map(map_scale)
    if colname==color_colname:
        dt_to_show = (dt_to_show
         .groupby([colname], dropna=False)
         .size()
         .reset_index(name='count')
        )
    else:
        dt_to_show = (dt_to_show
         .groupby([colname,color_colname], dropna=False)
         .size()
         .reset_index(name='count')
        )
        
    dt_to_show = (pd.DataFrame
     .from_dict(map_scale, orient='index', columns=[colname])
     .reset_index()
     .merge(dt_to_show,on=colname, how='left')
     .fillna(0))
    
    return plot_bars_percent_groupped(dt_to_show, colname, title, color_colname, 
                                       show_text, count_col, rot, legend=legend)


## Params

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Data

Carguemos los datos de los bloqueos y desbloqueos

In [5]:
bloq_desbloq = joblib.load('bloqueos_desbloqueos_obj.joblib')
bloq_desbloq['fecha_hora'] = pd.to_datetime(bloq_desbloq.FECHA_INGRESO.dt.strftime("%Y-%m-%d") + 
                                            ' ' +
                                            bloq_desbloq.HORA_INGRESO.dt.strftime("%H:%M:%S") )
bloq_desbloq.shape

(38134, 12)

Carguemos la tabla de novedades

In [6]:
novedades = joblib.load('novedades_obj.joblib')
novedades['fecha_hora'] = pd.to_datetime(novedades.FECHA_DESBLOQUEO.dt.strftime("%Y-%m-%d") + 
                                         ' ' +
                                         novedades.HORA_DESBLOQUEO.dt.strftime("%H:%M:%S") )
novedades.shape

(21935, 15)

Carguemos los datos de los colombianos en el exterior

In [22]:
df = pd.concat([
    pd.read_excel('MuestraKonecta.xlsx', sheet_name='Enero2020-Marzo2021'),
    pd.read_excel('MuestraKonecta.xlsx', sheet_name='Abril-Junio 2021 ')
])
# Convertamos a minutos por facilidad
df['duracion_seg'] = df['DURACION']
df['DURACION'] = df['DURACION']/60.0
df.shape

(9424, 27)

In [24]:
df2 = pd.concat([
    pd.read_excel('MuestraKonecta2.xlsx', sheet_name='Febrero 2022'),
    pd.read_excel('MuestraKonecta2.xlsx', sheet_name='Marzo 2022'),
    pd.read_excel('MuestraKonecta2.xlsx', sheet_name='Abril 2022'),
    pd.read_excel('MuestraKonecta2.xlsx', sheet_name='Mayo 2022'),
    pd.read_excel('MuestraKonecta2.xlsx', sheet_name='Junio 2022'),
    pd.read_excel('MuestraKonecta2.xlsx', sheet_name='Julio 2022')
])
# Convertamos a minutos por facilidad
df2['duracion_seg'] = df2['DURACION']
df2['DURACION'] = df2['DURACION']/60.0
df2.shape

(4415, 27)

Datos de Konecta con información de los asesores que contestan las llamdas, el código de desbloqueo y el tiempo que duró la llamada. Se tienen unos 9.4k de registros en un poco más de 17 meses (casi año y medio)

In [26]:
df.IDENTIFICACION.drop_duplicates().shape

(3925,)

Estos registros corresponden a 7939 clientes diferentes.

In [27]:
ips = joblib.load('ips_obj.joblib')

ITC sin país

In [28]:
ITC_Ips = joblib.load('ITC_Ips.joblib')
ITC_Ips['country'] = ITC_Ips.country.str.replace('\r','',regex=False)
ITC_Ips.loc[ITC_Ips.country=='','country'] = 'None'
# La siguiente corrección de restar 5 horas es necesaria debido a como subieron los datos
# de datetime a la LZ usando Sparky
# ITC_Ips['fecha_hora_llamada'] = ITC_Ips['fecha_hora_llamada'] - pd.DateOffset(hours=5)
ITC_Ips['fecha_hora_llamada'] = ITC_Ips['fecha_hora_llamada']

ITC etiquetado con país

In [29]:
ITC_etiquetado = (ITC_Ips
 .merge(df[['FECHA','IDENTIFICACION','DESCCOD']]
        .rename(columns={'FECHA':'fecha_hora_llamada',
                         'IDENTIFICACION':'identificacion',
                         'DESCCOD':'CodigoDesbloqueo'}),
        on=['identificacion','fecha_hora_llamada'], how='right')
)
ITC_etiquetado.shape

(1443892, 15)

# EDA

## Etiquetado Inicial

In [30]:
plot_bars_percent_firstk(df=df, colname='DESCCOD',
                         title='Frecuencia Tipos de Desbloqueos')

alt.LayerChart(...)

In [31]:
plot_bars_percent_firstk(df=df2, colname='DESCCOD',
                         title='Frecuencia Tipos de Desbloqueos')

alt.LayerChart(...)

Se puede ver que la gran mayoría de llamadas terminan en desbloqueo. Solo el 10.62% termina en "Desbloqueo no existoso" y el 1.08% es reconocido como "Detección suplatanción"

## Número de Llamadas en el Tiempo

### Por Día

In [32]:
base = alt.Chart(df.groupby([pd.Grouper(key='FECHA', freq='D')])
 .size()
 .reset_index(name='count')
).mark_line().encode(
    x = 'FECHA',
    y = alt.Y('count', scale=alt.Scale(domain=(0,140))),
    tooltip = ['FECHA', 'count']
).properties(
    title = 'Número de Llamadas por Día'
)

base + base.transform_loess('FECHA', 'count', bandwidth = 0.05).mark_line(size=1, color='red') 

alt.LayerChart(...)

In [33]:
base = alt.Chart(df2.groupby([pd.Grouper(key='FECHA', freq='D')])
 .size()
 .reset_index(name='count')
).mark_line().encode(
    x = 'FECHA',
    y = alt.Y('count', scale=alt.Scale(domain=(0,140))),
    tooltip = ['FECHA', 'count']
).properties(
    title = 'Número de Llamadas por Día'
)

base + base.transform_loess('FECHA', 'count', bandwidth = 0.05).mark_line(size=1, color='red') 

alt.LayerChart(...)

Hubo un pico de llamadas de colombianos en el exterior entre Enero y Mayo del 2021. Sin embargo, el número de llamdas se ha ido estabilizando un poco en los últimos meses.

### Por Mes

In [34]:
alt.Chart(df[df.FECHA < '2021-06-01'].groupby([pd.Grouper(key='FECHA', freq='M')])
 .size()
 .reset_index(name='count')
).mark_line().encode(
    x = 'FECHA',
    y = alt.Y('count'),
    tooltip = ['FECHA', 'count']
).properties(
    title = 'Número de Llamadas por Mes'
)

alt.Chart(...)

In [38]:
alt.Chart(df2[df2.FECHA < '2022-08-01'].groupby([pd.Grouper(key='FECHA', freq='M')])
 .size()
 .reset_index(name='count')
).mark_line().encode(
    x = 'FECHA',
    y = alt.Y('count'),
    tooltip = ['FECHA', 'count']
).properties(
    title = 'Número de Llamadas por Mes'
)

alt.Chart(...)

Esta gráfico es parecido al anterior pero por mes en vez de por día. Se excluye los datos de Junio para no tener datos parciales de éste mes.

## Duración Llamadas

### En el Tiempo

In [39]:
df_to_plot = (df.groupby([pd.Grouper(key='FECHA', freq='M')])
 .agg(duracion_mean=('DURACION','mean'),
      duracion_std=('DURACION','std'),
      duracion_cnt=('DURACION','size'))
 .assign(mean_std = lambda x: x.duracion_std / np.sqrt(x.duracion_cnt))
 .assign(duracion_min = lambda x: x.duracion_mean - 2*x.mean_std)
 .assign(duracion_max = lambda x: x.duracion_mean + 2*x.mean_std)
 .reset_index()
 .round(2)
)

line = alt.Chart(df_to_plot).mark_line().encode(
    x = 'FECHA',
    y = alt.Y('duracion_mean'),
    tooltip = ['FECHA', 'duracion_mean', 'mean_std']
).properties(
    title = 'Promedio e Intervalo de Confianza sobre la media en Duración (Minutos)'
)

area = alt.Chart(df_to_plot).mark_area(opacity=0.3, color='#57A44C').encode(
    x = 'FECHA',
    y = alt.Y('duracion_min'),
    y2 = alt.Y2('duracion_max')
)

area + line + line.transform_regression('FECHA', 'duracion_mean').mark_line(color='red')

alt.LayerChart(...)

In [40]:
df2_to_plot = (df2.groupby([pd.Grouper(key='FECHA', freq='M')])
 .agg(duracion_mean=('DURACION','mean'),
      duracion_std=('DURACION','std'),
      duracion_cnt=('DURACION','size'))
 .assign(mean_std = lambda x: x.duracion_std / np.sqrt(x.duracion_cnt))
 .assign(duracion_min = lambda x: x.duracion_mean - 2*x.mean_std)
 .assign(duracion_max = lambda x: x.duracion_mean + 2*x.mean_std)
 .reset_index()
 .round(2)
)

line = alt.Chart(df2_to_plot).mark_line().encode(
    x = 'FECHA',
    y = alt.Y('duracion_mean'),
    tooltip = ['FECHA', 'duracion_mean', 'mean_std']
).properties(
    title = 'Promedio e Intervalo de Confianza sobre la media en Duración (Minutos)'
)

area = alt.Chart(df2_to_plot).mark_area(opacity=0.3, color='#57A44C').encode(
    x = 'FECHA',
    y = alt.Y('duracion_min'),
    y2 = alt.Y2('duracion_max')
)

area + line + line.transform_regression('FECHA', 'duracion_mean').mark_line(color='red')

alt.LayerChart(...)

Extrañamente, parece que hay una leve tendencia de incremento en el promedio de la duración de las llamadas en el tiempo. El incremento en la variabilidad en Mayo, Junio y Julio del 2020, se deben a las pocas llamadas durante esos meses como se ve en la linea de tendencia roja.

### Por Tipo de Desbloqueo

In [41]:
y_axis = alt.Axis(
    title='Tipo Desbloqueo',
    offset=5,
    ticks=False,
    minExtent=60,
    domain=False
)

def q2_5(x):
    return x.quantile(0.025)

def q97_5(x):
    return x.quantile(0.975)

df_plot = (df.groupby(['DESCCOD'])
 .agg(duracion_mean=('DURACION','mean'),
      duracion_std=('DURACION','std'),
      duracion_min=('DURACION',q2_5),
      duracion_max=('DURACION',q97_5))
 .reset_index() 
 .round(2)
)

order_desbloque = df_plot.sort_values('duracion_mean', ascending=False).DESCCOD.tolist()

base = alt.Chart(df_plot).mark_rule().encode(
    y = alt.Y('DESCCOD', axis=y_axis, sort = order_desbloque),
    x = alt.X('duracion_min'),
    x2 = 'duracion_max',
    tooltip = ['DESCCOD', 'duracion_min', 'duracion_mean', 'duracion_max']
).properties(
    title = 'Media e Intervalo de Confianza del 95% Duración (Min.) por Tipo Desbloqueo'
)

tick = base.mark_point(
    filled=True,
    color='black',
    size=20,  # controls width of tick.
).encode(
    x='duracion_mean',
    y= alt.Y('DESCCOD', axis=y_axis, sort = order_desbloque)
)

base.encode(color = alt.Color('DESCCOD', legend=None)) + tick

alt.LayerChart(...)

In [42]:
y_axis = alt.Axis(
    title='Tipo Desbloqueo',
    offset=5,
    ticks=False,
    minExtent=60,
    domain=False
)

def q2_5(x):
    return x.quantile(0.025)

def q97_5(x):
    return x.quantile(0.975)

df2_plot = (df2.groupby(['DESCCOD'])
 .agg(duracion_mean=('DURACION','mean'),
      duracion_std=('DURACION','std'),
      duracion_min=('DURACION',q2_5),
      duracion_max=('DURACION',q97_5))
 .reset_index() 
 .round(2)
)

order_desbloque = df2_plot.sort_values('duracion_mean', ascending=False).DESCCOD.tolist()

base = alt.Chart(df2_plot).mark_rule().encode(
    y = alt.Y('DESCCOD', axis=y_axis, sort = order_desbloque),
    x = alt.X('duracion_min'),
    x2 = 'duracion_max',
    tooltip = ['DESCCOD', 'duracion_min', 'duracion_mean', 'duracion_max']
).properties(
    title = 'Media e Intervalo de Confianza del 95% Duración (Min.) por Tipo Desbloqueo'
)

tick = base.mark_point(
    filled=True,
    color='black',
    size=20,  # controls width of tick.
).encode(
    x='duracion_mean',
    y= alt.Y('DESCCOD', axis=y_axis, sort = order_desbloque)
)

base.encode(color = alt.Color('DESCCOD', legend=None)) + tick

alt.LayerChart(...)

Las llamadas más largas son aquellas que necesitan soportes, seguidas por las llamadas de desbloqueo exitoso.

In [43]:
alt.Chart(df
 .groupby(['DESCCOD'])
 .DURACION
 .sum()
 .reset_index(name='MinutosTotales')
 .assign(percent = lambda x: 100*x.MinutosTotales/x.MinutosTotales.sum())
 .round(2)
).mark_bar().encode(
    y = alt.Y('DESCCOD', sort='-x'),
    x = 'MinutosTotales',
    color = alt.Color('DESCCOD', legend=None),
    tooltip = ['DESCCOD', 'MinutosTotales', 'percent']
).properties(
    title = 'Total Número de Minutos por Desbloqueo'
)

alt.Chart(...)

In [44]:
alt.Chart(df2
 .groupby(['DESCCOD'])
 .DURACION
 .sum()
 .reset_index(name='MinutosTotales')
 .assign(percent = lambda x: 100*x.MinutosTotales/x.MinutosTotales.sum())
 .round(2)
).mark_bar().encode(
    y = alt.Y('DESCCOD', sort='-x'),
    x = 'MinutosTotales',
    color = alt.Color('DESCCOD', legend=None),
    tooltip = ['DESCCOD', 'MinutosTotales', 'percent']
).properties(
    title = 'Total Número de Minutos por Desbloqueo'
)

alt.Chart(...)

Esta gráfica nos muestra que el 77.6% de los mintutos en llamadas se dedica a los desbloqueos exitosos (2783 Horas). La clave para reducir los tiempos totales de las llamdas en COLEX es el de reducir el tiempo que requiere un asesor para identificar si el cliente es realmente un colombiano en el exterior.

### Por Día de la Semana y Hora

In [45]:
weekDays = ["Lunes","Martes","Miercoles","Jueves","Viernes","Sabado","Domingo"]

days = (pd.date_range(start=pd.to_datetime(df.FECHA.dt.strftime("%Y-%m-%d").min()), 
                    end=pd.to_datetime(df.FECHA.dt.strftime("%Y-%m-%d").max()), 
                    freq='D').astype(str).tolist())
hours = range(24)

# Producto Cartesiano entre días y horas
grid_ = np.array(list(product(*[days, hours])),dtype='str')
grid = (pd.DataFrame(grid_, columns = ['day','hour'])
        .assign(hour = lambda x: x.hour.astype(int)))

df_to_plot = (df
 .assign(day = lambda x: x.FECHA.dt.strftime("%Y-%m-%d"))
 .assign(hour = lambda x: x.FECHA.dt.hour)
 .groupby(['hour','day'])
 .size()
 .reset_index(name='count')
 .merge(grid, on=['day', 'hour'], how='right')
 .fillna(0)
 .assign(count = lambda x: x['count'].astype(int))
 .assign(dayofweek = lambda x: pd.to_datetime(x.day).dt.dayofweek.map(dict(zip(range(7),weekDays))))
 .groupby(['dayofweek', 'hour'])
 .agg(count_mean=('count','mean'))
 .reset_index()
)

alt.Chart(df_to_plot).mark_rect().encode(
    y = alt.Y('dayofweek:O', sort=weekDays),
    x = 'hour:O',
    color = 'count_mean:Q',
    tooltip=['dayofweek', 'hour', 'count_mean']
).properties(
    title = 'Mapa de Calor del Promedio de Llamadas por Día de la Semana y Hora'
)

alt.Chart(...)

Parece haber una tendencia marcada de llamadas el domingo durante todo el día (8 a 22) y al principio de la semana

### Por Día de la Semana

In [46]:
weekDays = ["Lunes","Martes","Miercoles","Jueves","Viernes","Sabado","Domingo"]

# listemos todos los días en el intervalo
idx = pd.date_range(start=pd.to_datetime(df.FECHA.dt.strftime("%Y-%m-%d").min()), 
                    end=pd.to_datetime(df.FECHA.dt.strftime("%Y-%m-%d").max()), 
                    freq='D')
# Hagamos el calculo del promedio llenando los días sin observaciones con cero
df_to_plot = (df
 .assign(day = lambda x: pd.to_datetime(x.FECHA.dt.strftime("%Y-%m-%d")))
 .groupby(['day'])
 .size()
 .reindex(idx, fill_value=0)
 .reset_index(name='count')
 .rename(columns={'index':'FECHA'})
 .assign(dayofweek = lambda x: x.FECHA.dt.dayofweek.map(dict(zip(range(7),weekDays))))
 .groupby(['dayofweek'])
 .agg(count_mean=('count','mean'),
      count_std=('count','std'),
      count_cnt=('count','size'))
 .reset_index()
 .assign(mean_std = lambda x: x.count_std / np.sqrt(x.count_cnt))
 .assign(count_min = lambda x: x.count_mean - 2*x.mean_std)
 .assign(count_max = lambda x: x.count_mean + 2*x.mean_std)
 .round(2)
)

base = alt.Chart(df_to_plot).mark_rule().encode(
    y = alt.Y('dayofweek:N', sort=weekDays),
    x = 'count_min:Q',
    x2 = 'count_max:Q',
    tooltip = ['dayofweek', 'count_mean']
).properties(
    title = 'Promedio de llamadas por día de la semana'
)

tick = base.mark_point(
    filled=True,
    color='black',
    size=20,  # controls width of tick.
).encode(
    x='count_mean',
    y= alt.Y('dayofweek', sort = order_desbloque)
)

base.encode(color = alt.Color('dayofweek', legend=None)) + tick

alt.LayerChart(...)

## Desbloqueos por Asesor/a

### Número

Cómo se ve en esta gráfica, Domingos, Lunes y Martes son los días con más altos promedios con entre 16 y 25 llamadas por día, mientras que Miercoles, Jueves y Viernes tienen un promedio muy parecido y sabado es el día con menos promedio de llamadas.

In [47]:
plot_bars_percent_firstk(df=df, colname='ASESOR',
                         title='Número de Llamadas Atendidas por Asesor en Intervalo Analizado',
                         show_text=False)

alt.Chart(...)

In [48]:
plot_bars_percent_firstk(df=df2, colname='ASESOR',
                         title='Número de Llamadas Atendidas por Asesor en Intervalo Analizado',
                         show_text=False)

alt.Chart(...)

Un total de 69 asesores han atendido las llamadas de Colombianos en el exterior en el periodo analizado. El asesor que más llamadas a recibido es paavelas con 333, sin embargo, no llega ni al 4% del total de todas las llamadas.

### Tasa Tipo Desbloqueo

In [49]:
alt.Chart(df[df.DESCCOD!='Desbloqueo pendiente de validacion ']
 .groupby(['ASESOR','DESCCOD'])
 .agg(count=('FECHA','size'))
 .assign(total = lambda x: x.groupby(level='ASESOR')['count'].transform(lambda z: z.sum()))
 .assign(percent = lambda x: x['count']/x['total'])
 .reset_index()
).mark_point().encode(
    x = 'total',
    y = 'percent',
    color = 'DESCCOD',
    tooltip = ['ASESOR' ,'DESCCOD', 'count', 'total', 'percent']
).properties(
    title = 'Porcentaje Tipo Desbloqueo por Asesor'
)

alt.Chart(...)

In [50]:
alt.Chart(df2[df2.DESCCOD!='Desbloqueo pendiente de validacion ']
 .groupby(['ASESOR','DESCCOD'])
 .agg(count=('FECHA','size'))
 .assign(total = lambda x: x.groupby(level='ASESOR')['count'].transform(lambda z: z.sum()))
 .assign(percent = lambda x: x['count']/x['total'])
 .reset_index()
).mark_point().encode(
    x = 'total',
    y = 'percent',
    color = 'DESCCOD',
    tooltip = ['ASESOR' ,'DESCCOD', 'count', 'total', 'percent']
).properties(
    title = 'Porcentaje Tipo Desbloqueo por Asesor'
)

alt.Chart(...)

En general, hay unos comportamientos bastante estables entre los asesores. Sin embargo, hay algunos casos atípicos como por ejemplo el/la asesor/a "miljaram" que con 7 "desbloqueos no exitosos" de los 8 clientes atendidos. Por otro lado, "vanlopez" ha tenido 33 "desbloqueo con soportes" de los 77 clientes atendidos. Por último, están los casos de "jufebedo" con 21 "desbloqueos no existosos" de los 45 clientes atendidos y "vipelaez" con 77 "desbloqueos no existosos" de los 216 clientes atendidos.

En conclusión, aunque hay alguna uniformidad en los datos por asesor, existes algunos casos que llaman la atención.

Finalmente, los casos de "Deteccion suplantancion" estan bastante repartidos entre todos los asesores. al asesor/a "jucflore" le han tocado 9 casos de suplantación detectada entre 99 clientes atendidos.

In [53]:
df.IDENTIFICACION.value_counts().head(5)

1111111111    13
98508536       7
1026263574     7
45551345       6
5556615        6
Name: IDENTIFICACION, dtype: int64

Como era de esperarse, salen algunos documentos con multiples llamdas. Veamos esos casos.

In [54]:
alt.Chart(df[df.IDENTIFICACION==98508536]
).mark_bar().encode(
    x = 'FECHA',
    y = 'count(FECHA)',
    color = 'DESCCOD',
    tooltip = ['FECHA','count(FECHA)' ,'DESCCOD', 'ASESOR']
).properties(
    title = 'Llamadas para Cliente con Documento 98508536'
)

alt.Chart(...)

Estos son las 7 llamadas del cliente con cédula número 98508536. Se puede ver que es una mezcla de diferentes códigos de desbloqueo en las diferentes llamadas. De hecho, en cada llamada fue atendido por un asesor diferente. Estos casos son los más dificiles de calificar dado que normalmente, presentan más ambiguedad.

## Desbloqueos Primera Llamada

In [55]:
exitosos_primera_llamada = (df
 .assign(num_llamadas = df.groupby('IDENTIFICACION').FECHA.transform(lambda x: len(x)))
 .query("(num_llamadas==1) & (DESCCOD=='Desbloqueo exitoso')")
)
exitosos_primera_llamada.shape

(5303, 28)

In [57]:
exitosos_primera_llamada2 = (df2
 .assign(num_llamadas = df2.groupby('IDENTIFICACION').FECHA.transform(lambda x: len(x)))
 .query("(num_llamadas==1) & (DESCCOD=='Desbloqueo exitoso')")
)
exitosos_primera_llamada2.shape

(2378, 28)

Se tienen 5303 casos en el periodo estudiado donde se concedió un desbloqueo exitoso en la única llamada que nos aparce en el conjunto de datos.

In [58]:
alt.Chart(exitosos_primera_llamada[exitosos_primera_llamada.FECHA < '2021-06-01']
 .groupby(pd.Grouper(key='FECHA', freq='M'))
 .size()
 .reset_index(name='count')
).mark_bar().encode(
    x = 'FECHA',
    y = 'count',
    tooltip = ['FECHA', 'count']
).properties(
    title = 'Número de Clientes Desbloqueados Exitosamente en Primera Llamada'
)

alt.Chart(...)

Se ve que son un volumen considerable y que son clientes objetivo a etiquetar cómo colombiano en el exterior. En teoría, estos casos deberían de ser los casos más fáciles de calificar. Sin embargo, es un hipotesis a verificar.

Revisemos el promedio y el intervalo de confianza del 95%

In [59]:
print(round(exitosos_primera_llamada.DURACION.mean(),2),
     (round(q2_5(exitosos_primera_llamada.DURACION),2),
      round(q97_5(exitosos_primera_llamada.DURACION),2)))

23.83 (1.98, 53.31)


Tenemos que el 95% de las llamadas están enrte 2 y 53 minutos aproximadamente con una media de 23.83 minutos.

In [60]:
exitosos_primera_llamada.sort_values(by = 'FECHA', ascending=False)

,FECHA,CODIGO_ACTIVIDAD,COD_PRODUCTO,TIPO_IDENTIFICACION,IDENTIFICACION,NOMBRE,ASESOR,NOMBRES_ASESOR,APELLIDOS_ASESOR,EXTENSION,TELEFONO_IN,TELEFONO_OUT,REGLANEGOCIO,COD_EMP,DURACION,CONNID,REF_ID,TRANSFER,AG_TRANSFER,DNIS,CLAVE,TIPO_LLAMADA,CTI_TIPORI,GSW_CAMPAIGN_NAME,CTI_FECHAC,DESCCOD,duracion_seg,num_llamadas
2398,2021-06-10 20:36:36,107,CCE,1,78715286,MAYKOL ENRIQUE BUELVAS HERNANDEZ,yszapata,Yamid Stiven,Zapata Hernandez,81619,720786,85596,MR,193,7.133333,00000314dcd47eae,02077DH9KK8BF6H7384H5B5AES004E6A,,CONFERENCIA DESDE_ LES BASICA CON_ luis_beni...,702450,S,2,C,|037c0309d7c65885,,Desbloqueo exitoso,428,1
2397,2021-06-10 20:25:01,107,CCE,1,7632725,ANGELO HELI SIERRA BLANCO,vipelaez,Viviana Maria,Pelaez,81614,720741,415728,MR,193,3.166667,037c0309d7c65403,02077DH9KK8BF6H7384H5B5AES004E3R,,CONFERENCIA DESDE_ LES BASICA CON_ andres_su...,702450,N,4,C,|037c0309d7c65403,,Desbloqueo exitoso,190,1
2395,2021-06-10 18:48:16,107,CCE,1,1130681718,MARIA CAMILA SILVA PALOMINO,jodurang,Jorge,Durango,85002,720233,417065,MR,193,32.800000,00000314dcd4424d,02077DH9KK8BF6H7384H5B5AES004CUN,T| |53381,CONFERENCIA DESDE_ LES BASICA CON_ maria_mor...,702450,S,2,C,|037c0309d7c60022,,Desbloqueo exitoso,1968,1
2394,2021-06-10 18:23:06,107,CCE,1,71394670,JORGE ELIECER ZAPATA BERMUDEZ,srville,Sebastian,Ramirez Villega,81608,720818,415722,MR,193,17.750000,00000314dcd43251,02077DH9KK8BF6H7384H5B5AES004CL4,C|CX|56402,CONFERENCIA DESDE_ LES BASICA CON_ yuri_agud...,56402,S,2,C,|037c0309d7c5e25e,,Desbloqueo exitoso,1065,1
2393,2021-06-10 18:16:13,107,CCE,1,43077414,ANA LUCIA TORRES VELEZ,jumarang,Juan Manuel,Arando Lopera,81644,14019998139,417028,XL,193,3.433333,037c0309d7c5d98b,01ISNM0PQG85976CF0LH9B5AES0HMRFJ,,CONFERENCIA DESDE_ SERVICIO BASICO CON_ caro...,53394,V,4,C,|037c0309d7c5d98b,,Desbloqueo exitoso,206,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,2020-01-04 04:23:27,107,CCE,1,39614114,CARMEN ROSA SABOGAL ARDILA,dcatano,Daniela,Catano Lopera,81610,2222222,89279,MR,193,29.883333,000002ea985ebb6f,000002ea985ebb6f,T| |53381,CONFERENCIA DESDE_ VIRTUALES COLOMBIANOS EN E...,1010,S,2,C,|037c02e0d3347f30,,Desbloqueo exitoso,1793,1
5,2020-01-03 15:17:25,107,CCE,1,41885669,MARIA CLARIBEL LEIVA RUBIANO,sanhinca,Sandra Yuliana,Hincapie Arboleda,81653,1111113,85045,XK,193,24.250000,037c02e0d3324e1b,00Q0L545A0EUHEN3F0LH9B5AES053BBL,T| |53381,CONFERENCIA DESDE_ SERVICIO BASICO CON_ mayr...,1010,S,3,C,|037c02e0d3326076,,Desbloqueo exitoso,1455,1
3,2020-01-02 22:01:51,107,CCE,1,1232595467,JOY NIKOLAY LOPERA CARDONA,jaquicen,Jeison Alexander,Quiceno Quintero,81605,584162462929,81616,MR,193,26.750000,037c02e0d32e2b0f,00Q0L545A0EUHEN3F0LH9B5AES04V3Q4,,CONFERENCIA DESDE_ VIRTUALES COLOMBIANOS EN E...,1010,S,3,C,|037c02e0d32e2e5c,,Desbloqueo exitoso,1605,1
1,2020-01-01 17:41:19,107,CCE,1,1018448735,LINA XIMENA NIÃ‘O HERRERA,paavelas,Paula Andrea,Velasquez Giron,81652,4407308661980,85383,MF,193,6.266667,037c02e0d32825fd,00Q0L545A0EUHEN3F0LH9B5AES04OA8P,T| |53381,CONFERENCIA DESDE_ VIRTUALES COLOMBIANOS EN E...,1010,S,3,C,|037c02e0d3282762,,Desbloqueo exitoso,376,1


## Desbloqueos y Desbloqueos

En los datos de KONECTA, tenemos los desbloqueos de colombianos en el exterior sin embargo, no están los bloqueos que llevaron a la llamada. En esta base de datos, tenemos tanto los bloqueos como los desbloqueos de las cedulas que aparecen en las llamadas de colombianos en el exterior.

In [61]:
plot_bars_percent_firstk(df=bloq_desbloq, colname='TIPO_BLOQUEO', k=10,
                         title='Top 10 Frecuencia de Tipos de Des/Bloqueos')

alt.LayerChart(...)

La mayoria de bloqueos es por **Bloqueo Clave Dinamica** con el **50.89%** y **Bloqueo S Clave Dinamica** con el **20.91%**. Finalmente están los **Desbloqueo** con el **15.08%**.

## Pruebas

In [ ]:
cedula = exitosos_primera_llamada[exitosos_primera_llamada.DESCCOD=='Desbloqueo exitoso'].sample(1).IDENTIFICACION.tolist()[0]
cedula = 19073369
# cedula

In [ ]:
llamada = df[df.IDENTIFICACION==cedula][['IDENTIFICACION','TIPO_IDENTIFICACION','FECHA','DURACION','DESCCOD','ASESOR','AG_TRANSFER']].assign(IDENTIFICACION = lambda x: x.IDENTIFICACION.astype(str))
llamada

In [ ]:
bd = (bloq_desbloq[(bloq_desbloq.ID_CLIENTE==f'{cedula}')]
              [['ID_CLIENTE','fecha_hora','TIPO_BLOQUEO', 'SALDO_DISPONIBLE', 'OBSERVACIONES']]
      .sort_values('fecha_hora', ascending=False))
display(HTML(bd.to_html()))

In [ ]:
nov = (novedades[(novedades.ID_CLIENTE==f'{cedula}')]
              [['ID_CLIENTE','fecha_hora','CAUSA_DESMARCACION','OBSERVACIONES']]
       .rename(columns = {'OBSERVACIONES':'OBSERVACIONES_NOV', 
                          'fecha_hora': 'fecha_hora_nov',
                          'ID_CLIENTE':'ID_CLIENTE_NOV'})
      .sort_values('fecha_hora_nov', ascending=False))
display(HTML(nov.to_html()))

In [ ]:
datos_tbl_inf_prod = (llamada[['IDENTIFICACION', 'FECHA']]
 .merge(bd,
        left_on='IDENTIFICACION',
        right_on='ID_CLIENTE',
        how='left')
 .assign(time_dif = lambda x: ((x.fecha_hora - x.FECHA) / pd.Timedelta(seconds=60)))
 .assign(SALDO_DISPONIBLE = lambda x: x.SALDO_DISPONIBLE.astype(str))
 .round(2)
 .query("(time_dif > 0) & (time_dif < 60*24)")
 .sort_values(by='time_dif')
 .groupby(['FECHA'])
 .head(1)
 .sort_values(by='FECHA')
)

datos_tbl_novedades = (llamada[['IDENTIFICACION', 'FECHA']]
 .merge(nov,
        left_on='IDENTIFICACION',
        right_on='ID_CLIENTE_NOV',
        how='left')
 .assign(time_dif_nov = lambda x: ((x.fecha_hora_nov - x.FECHA) / pd.Timedelta(seconds=60)))
 .round(2)
 .query("(time_dif_nov < 10) & (time_dif_nov >-10)")
 .sort_values(by='time_dif_nov')
 .groupby(['FECHA'])
 .head(1)
 .sort_values(by='FECHA')
)

datos = (llamada
    .merge(datos_tbl_inf_prod,
           on=['IDENTIFICACION','FECHA'],
           how='left')
    .merge(datos_tbl_novedades,
           on=['IDENTIFICACION','FECHA'],
           how='left')
    .sort_values(by='FECHA', ascending=True)
)

from IPython.core.display import display, HTML
html_block = datos.to_html()
display(HTML(html_block))

In [ ]:
(ITC_etiquetado[(ITC_etiquetado.identificacion==cedula) & ITC_etiquetado.canal.str.strip().isin(['PCR'])]
 .groupby(['fechatrn','canal','cdgtrn', 'nomdisp','country', 'campo5','vlrtran'])
 .agg(valor_comprado = ('vlrtran','sum'),
      nro_trns = ('vlrtran','size'))
 .reset_index().sort_values('fechatrn',ascending=False)
 .sort_values(by='fechatrn', ascending=False))[:50]

In [ ]:
(ITC_etiquetado[(ITC_etiquetado.identificacion==cedula)]
 .groupby(['fechatrn','canal', 'nomdisp','country', 'campo5','vlrtran'])
 .agg(valor_comprado = ('vlrtran','sum'),
      nro_trns = ('vlrtran','size'))
 .reset_index().sort_values('fechatrn',ascending=False)
 .sort_values(by='fechatrn', ascending=False))[:50]

In [ ]:
(ITC_etiquetado[(ITC_etiquetado.identificacion==cedula) & (ITC_etiquetado.canal.str.strip().isin(['SAI','ABB']))]
 .sort_values('fechatrn',ascending=False))

## Geolocalizando IPs

In [ ]:
# !pip install pygeoip

In [ ]:
import pygeoip
gi = pygeoip.GeoIP('maxmind4country.dat')
gi.country_name_by_addr('68.173.180.181')

In [ ]:
import re
a = re.match(r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.$",'191.156.154.')
if a!=None:
    print('match')
else:
    print('no match')

In [ ]:
import re
def ip2country(x):
    country = 'None'
    a = re.match(r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.$",x)
    if a!=None:
        x=x+'001'
    try:
        country = gi.country_code_by_addr(x)
    except:
#         print(x)
        counrty = 'None'
        
    return country
        
ips['country'] = 'None'
ips.loc[ips.canal.str.strip().isin(['APP','SVP']), 'country'] = (ips[ips.canal.str.strip().isin(['APP','SVP'])]
 .nomdisp.str.strip().replace('^0+','', regex=True)
 .map(lambda x: ip2country(x))
)
ips

In [ ]:
ips.shape

In [ ]:
# from sparky_bc import Sparky
# sp = Sparky('mausoto', password = '')

In [ ]:
# sp.subir_df(ips, 
#             nombre_tabla='proceso_seguridad_externa.colex_ips_to_country', 
#             modo = 'overwrite', 
#             multiLine = True, 
#             escape = '"', 
#             spark_options = '--num-executors 2 --executor-memory 1GB --conf spark.dynamicAllocation.enabled=false')

## ITC con IPs

In [ ]:
ITC_etiquetado.info()

### Por País

In [ ]:
plot_bars_percent_firstk(df=ITC_Ips[ITC_Ips.country!='None'], colname='country', k=20)

La mayoría de IPs en el exterior son desde Estados Unidos, seguido por Colombia, España y Otros.

### Por Canal

In [ ]:
plot_bars_percent_firstk(df=ITC_Ips, colname='canal', k=5,
                         title='Top 5 Frecuencia de Canales')

Afortunadamente, la mayoría de transacciones son por APP y SVP, lo que tiene sentido, sabiendo que son clientes que en los últimos días estaban en el exterior.

In [ ]:
(ITC_Ips
 .groupby(['identificacion','fecha_hora_llamada'])
 .size()
 .reset_index(name='count')
 .merge(df[['FECHA','IDENTIFICACION','DESCCOD']]
        .rename(columns={'FECHA':'fecha_hora_llamada',
                         'IDENTIFICACION':'identificacion',
                         'DESCCOD':'CodigoDesbloqueo'})
        .assign(fecha_hora_llamada = lambda x: x.fecha_hora_llamada + pd.DateOffset(hours=5)),
        on=['identificacion','fecha_hora_llamada'], how='right')
 .fillna(0)
 .assign(count = lambda x: x['count'].astype(int))
 .assign(tiene_ITC = lambda x: (x['count']>0).astype(int))
 .groupby(['tiene_ITC'])
 .size()
 .reset_index(name='freq')
)

Realmente son muy pocos los clientes que llaman por un bloqueo y que no tienen movimientos en el ITC en los últimos 30 días.

In [ ]:
alt.Chart(ITC_Ips
 .groupby(['identificacion','fecha_hora_llamada'])
 .size()
 .fillna(0)
 .reset_index(name='count')
 .groupby(['count'])
 .size()
 .reset_index(name='freq')
).mark_bar().encode(
    x = alt.X('count', scale=alt.Scale(type='log')),
    y = 'freq',
    tooltip = ['count','freq'],
    color=alt.condition(
        alt.datum.count > 10,
        alt.value("steelblue"),
        alt.value("orange"))
).properties(
    title = 'Número de transacciones ITC últimos 30 días'
)

De hecho, se ve que la mayoría de las llamadas son de clientes que tienen más de 10 datos en el ITC

## Transacciones Físicas

Hasta ahora, sólo se han explorado variables relacionadas a transacciones virtuales en los canales APP y SVP. Sin embargo, no se han integrado variables relacionadas a transacciones físicas localizables.

Según la informacion recaudada, es posible identificar si una transacción del ITC fue presencial y si ella fue nacional o internacional usando las variables canal, cdgtrn y campo5.

La primera fuente de información de transacciones presenciales son las del canal **ABB** que indica transacciones en cajeros automáticos propios de Bancolombia en Colombia. Por otro lado estan las transacciones en el canal **SAI** que son transacciones en sucursales físicas.

In [ ]:
(ITC_Ips[ITC_Ips.canal.str.strip().isin(['ABB','SAI'])]
 .groupby(['canal'],dropna=False)
 .size()
 .reset_index(name='count')
)

Por otr lado, estan las transacciones con tarjeta que están relacionados a los canales: 

- **MII**: Punto de Venta Redeban
- **RDB**: Cajero Redeban
- **PRD**: Punto de Venta Redeban
- **MCI**: Master International
- **PAX**: Punto de Venta AMEX
- **PCM**: Punto de Venta Credibanco
- **PMR**: Punto de Venta Redeban
- **PPM**: Punto de Venta Mastercard
- **PCR**: Punto de Venta Visa


In [ ]:
df_to_show = ITC_Ips[ITC_Ips.canal.str.strip().isin(['MII','RDB','PRD','MCI','PAX','PCM','PMR','PPM','PCR'])]
plot_bars_percent_firstk(df=df_to_show, colname='canal',
                         title='Canales de Interés Compras')

Sin embargo, no todas las transacciones en estos canales son presenciales. En particular, las compras en los códigos **226**, **426** y **526** son **compras no presenciales** que se deben filtrar.

In [ ]:
df_to_show = (ITC_Ips[ITC_Ips.canal.str.strip().isin(['MII','RDB','PRD','MCI','PAX','PCM','PMR','PPM','PCR']) &
         ~ITC_Ips.cdgtrn.isin([226, 426, 526])])
plot_bars_percent_firstk(df=df_to_show, colname='canal',
                         title='Canales de Interés con Filtro para Compras Físicas')

Finalmente, en este subconjunto de datos, podemos discriminar por el número de transacciones físicas si fueron nacionales o internacionales.

In [ ]:
nacional = (df_to_show[df_to_show.campo5.str.strip().isin(['N'])])
internacional = (df_to_show[df_to_show.campo5.str.strip().isin(['I'])])
nal_plot = plot_bars_percent_firstk(df=nacional, colname='canal',
                         title='Canales de Interés con Filtro para Compras Físicas Nacionales')
int_plot = plot_bars_percent_firstk(df=internacional, colname='canal',
                         title='Canales de Interés con Filtro para Compras Físicas Internacionales')
nal_plot | int_plot

Se puede observar que el canal **PRD** contiene la mayoria de transacciones nacionales, mientras que **RDB** y **PRD** la mayoría de transacciones internacionales.

## Codigos de Transacciones

Tratemos de entender cuales son los códigos de transferencia más importantes para discriminar entre desbloqueos exitosos y suplantaciones utilizando los datos. Para ello, creemos un simple modelo donde, para cada combinación de identificación y hora de llamada, tengamos todos los códigos de transferencia y si ese código estuvo presente en el último mes.

In [ ]:
data = (ITC_etiquetado[ITC_etiquetado.CodigoDesbloqueo.isin(['Desbloqueo exitoso','Deteccion suplantacion '])]
 .groupby(['identificacion','fecha_hora_llamada','CodigoDesbloqueo','cdgtrn'])
 .size()
 .reset_index(name='count')
 .assign(count = 1)
 .assign(cdgtrn = lambda x: x.cdgtrn.astype(int))
 .pivot(index=['identificacion','fecha_hora_llamada','CodigoDesbloqueo'], columns='cdgtrn',values='count')
 .fillna(0)
 .reset_index()
 .assign(CodigoDesbloqueo = lambda x: (x.CodigoDesbloqueo=='Deteccion suplantacion ').astype(int))
)
data

In [ ]:
y_ = data.CodigoDesbloqueo
x_ = data.drop(columns=['identificacion','fecha_hora_llamada','CodigoDesbloqueo'])
print(x_.shape,y_.shape)
print(y_.value_counts())

Son un total de 355 códigos y un total de 6.8K de datos

In [ ]:
m_ = BalancedRandomForestClassifier(n_estimators=2000, random_state=0, oob_score=True).fit(x_, y_)
preds_proba_train = m_.oob_decision_function_[:,1]
report_AUC(y_, preds_proba_train, data_set='training');

No tuvo un rendiiento muy malo el modelo. 0.77 es bastante aceptable. Parece indicar que hay poder predictivo en el tipo de códigos de transaccion que tienen los clientes para discriminar.

Ahora miremos cuales fueron los códigos más importantes:

In [ ]:
fi = rf_feat_importance(m_, x_.columns).sort_values(by='imp', ascending=False)
plot_fi(fi[fi.imp>0.0112])

Los códigos más importantes fueron:

- **167**: Recuperación de Saldos (TLF). No se que es.
- **2**: Consulta Saldo Deposito (AUD, DEB). Que es DEB?
- **260**: Consulta de Saldo de Tarjetas (Muchos canales incluyendo SVP, SVE y TLF).
- **171**: Consulta de tarjeta Débito (APP, SVP). Muy poquitos casos.
- **888**: Consulta del costo de la transacción. Los defraudadores no consultan esto.
- **410**: Anulación de compra. Es posible que sea solo cuando es real la compra o luego de un robo. No se muy bien.
- **3100** y **4100**: Este código es muy importante ya que es un enrolamiento APP y enrolamiento softoken personas. Son algunos códigos que deberían tener buen potencial pero quedaron bastante rezagados.
- **751**: código de cambio de topes en SVP. Tampoco quedó en los primeros lugares aunque se necesita para poder sacar el dinero.


In [ ]:
(ITC_etiquetado[ITC_etiquetado.cdgtrn.isin([167])]
 [['identificacion','fecha_hora_llamada','CodigoDesbloqueo']]
 .drop_duplicates()
 .CodigoDesbloqueo
 .value_counts())

30 de 61 (50% aproximadamente) suplantaciones usaron el código 167, mientras que solo 141 de 5144 (3.25%) de los desbloqueos exitosos.

In [ ]:
(ITC_etiquetado[ITC_etiquetado.cdgtrn.isin([2])]
 [['identificacion','fecha_hora_llamada','CodigoDesbloqueo']]
 .drop_duplicates()
 .CodigoDesbloqueo
 .value_counts())

42 de 61 (68%) suplantaciones usaron el código 2, mientras que solo 1080 de 5144 (21%%) de los desbloqueos exitosos.

In [ ]:
(ITC_etiquetado[ITC_etiquetado.cdgtrn.isin([167])]
 [['identificacion','fecha_hora_llamada','canal','CodigoDesbloqueo']]
 .drop_duplicates()
 .groupby(['CodigoDesbloqueo','canal'])
 .size()
 .sort_values(ascending=False)
)

Aqui vemos que todas las transacciones con código 167 se realizaron por el canal TLF.

### Revisión de Número trns por Cédula

In [ ]:
(ITC_etiquetado[(ITC_etiquetado.canal.str.strip().isin(['APP','SVP'])) & (ITC_etiquetado.identificacion.isin([42118877]))]
 .groupby(['identificacion','fecha_hora_llamada','cdgtrn'])
 .size()
 .reset_index(name='count')
 .sort_values(['identificacion','fecha_hora_llamada','count'], ascending=False))[:50]

## Limpieza Variable Objetivo

Utilizando los resultados de la primera iteración del modelo, me dí cuenta que hay mucho ruido en la variable objetivo. Sobretodo con los registros marcados como **Deteccion de Suplantacion**:

- Por un lado, parece que hay un asesor que ya no se encuentra en el banco (**jucflore**) que marcó llamadas de posible suplantación cómo suplantación confirmada.
- Por otro lado, hay varios casos de equivocación del asesor donde el registro quedo duplicado con **Detección de Suplantación** y otra marcación simultáneas.
- Finalmente, algunos casos donde las llamadas fueron marcadas como **Detección de Suplantación** y luego los comentarios de un bloqueo sucesivo nos indica que el cliente siempre fué el que llamó.


In [ ]:
llamadas_duplicadas = (df
 .groupby(['FECHA', 'IDENTIFICACION','ASESOR'])
 .agg(count = ('DESCCOD','count'))
 .query("count > 1")
 .sort_values('count', ascending=False)
 .reset_index()
)
llamadas_duplicadas

Se tienen 33 llamadas con doble y triple marcación. Esto es especialmente complicado para el modelo dado que vé dos regisros con exactamente las mismas variables del cliente pero con dos marcaciones distintas.

Veamos que asesores tuvieron estas dificultades.

In [ ]:
alt.Chart(llamadas_duplicadas
 .groupby(['ASESOR'])
 .size()
 .reset_index(name='count')).mark_bar().encode(
    y = alt.Y('ASESOR', sort='-x'),
    x = 'count',
    color = alt.Color('ASESOR', legend=None),
    tooltip = ['ASESOR', 'count']
).properties(
    title = 'Número de Llamadas con Marcación Múltiple'
)

### Ruido en Datos

Revisemos cuantos casos de marcación duplicada tenemos, donde una de las marcaciones es **Deteccion suplantacion**

In [ ]:
duplicados_con_suplantacions = (df.merge(llamadas_duplicadas,
         on=['FECHA', 'IDENTIFICACION','ASESOR'],
         how='inner')
 .query("DESCCOD == 'Deteccion suplantacion '")
 .loc[:, ['FECHA', 'IDENTIFICACION','ASESOR']]                               
 .merge(df,
        on=['FECHA', 'IDENTIFICACION','ASESOR'],
        how='inner')
)
duplicados_con_suplantacions.DESCCOD.value_counts()

Tenemos 5 casos donde, por un lado está marcado como **Deteccion suplantacion** y simulateneamente, tiene otra marcación. Por seguridad, eliminemos todos los datos duplicados.

En particular, se eliminan las llamadas de los clientes **40612622** (pred=0.95), **43827625** (pred=0.89) y **1053778928** (pred=0.73) dado que realmente fueron bloqueos exitosos o desbloqueo no existoso pero no una suplantación.

In [ ]:
df_clean = df[~df.FECHA.isin(llamadas_duplicadas.FECHA)]
df_clean.shape

De 9424 de los registros originales, nos quedan 9357.

In [ ]:
df_clean = df_clean[~df_clean.ASESOR.isin(['jucflore'])]
df_clean.shape

Quedan ahora 9223 registros quitando los registros de jucflore dado que nos son registros muy confiables.

Se eliminan los siguientes registros de **Deteccion suplantacion** según validación con Diana Carolina Yepes Angel:

- **12203773** (pred=0.95): El mismo día fue marcado con suplantación y desbloqueado con chequeo de IUVI.
- **7448141** (pred=0.9033): El asesor no dejó comentarios en base de datos para entender si fue realmente una suplantación
- **80888357** (pred=0.88): Caso de suplantación no confirmada por diasuaza el 26 de Enero que luego desbloqueada el 30.
- **88237859** (pred=0.87): jucflore marco como suplantación un caso no confirmado que luego se vió que era legítimo. 
- **85452242** (pred=0.825): Marcacion como suplantacion en llamada de 5 minutos y una hora despues se desbloquea.
- **1094944184** (pred=0.7567): Cliente cuelga enojado pero asesor llama a la mama que confirma que el cliente si está en Chile.
- **1053831270** (pred=0.7233): Cliente marcado con suplantación el 28 de Marzo pero luego desbloqueado el 8 de Abril. Se chequean las IPs y son las mismas en el periodo del 28 de Marzo al 8 de Abril.
- **34604711** (pred=0.6783): Cliente marcado como suplantación el 26 de Septiembre 2020. Luego desbloqueado el 3 de Octubre. IPs siempre en España.
- **1030531247** (pred=0.6567): Cliente con bastante transaccionalidad en Australia. Marcado como suplantación por no pasar filtros pero no es suplantación confirmada. Bloqueo por compra con tarjeta de crédito.
- **21486081** (pred=0.6067): Caso marcado cómo suplantacion por jucflore pero desbloqueado al dia siguiente. Todas las IPs desde Italia.
- **41454664** (pred=0.6067): Marcado con suplantacion el 8 de Dic del 2020. Desbloqueado luego en Enero donde dice que hace 5 años con documentos. No viene a Colombia hace 5 años y lo soporta las IPs.
- **1006306287** (pred=0.53): Cliente suena sospechoso llamando desde España. Manda varias veces cedula borrosa y sin datos necesarios. Dice que llama del locutorio (sitio de llamadas en España). Luego lo desbloquean.
- **31481215** (pred=0.41): Cliente son sospecha de suplantación pero no es confirmada. Llama varias veces. Bloqueado por remesa que él mismo envió. Luego es desbloqueado.
- **5018683877** (pred=0.53): Loggeos desde USA y Venezuela. Dice que el cliente está en USA pero hermano en Venezuela. Marcación de suplantacion sin comentario asociado.
- **80030197** (pred=0.4267): Marcado como suplantacion el 19/01/2021. Cliente se enoja. Dice que está en españa. Comentarios dicen que no tiene perfil transaccional pero si veo entrada desde España el 13 de Enero. Luego lo desbloquean el 10 de Febrero.
- **65813141**: Esta etiquetado como suplantador pero fue desloqueado con exito según mensaje y tiene una transacción física en el exterior.



In [ ]:
cedulas_drop_suplantacion = [
    12203773,
    7448141,
    80888357,
    88237859,
    85452242,
    1094944184,
    1053831270,
    34604711,
    1030531247,
    21486081,
    41454664,
    1006306287,
    31481215,
    5018683877,
    80030197,
    65813141
]
df_clean = df_clean[~(df_clean.IDENTIFICACION.isin(cedulas_drop_suplantacion) & 
                     (df_clean.DESCCOD == 'Deteccion suplantacion '))]
df_clean.shape

Finalmente quedamos con 9215 registros.

### Suplantaciones Confirmadas

Es muy importante resaltar los casos con suplantación confirmada y que el modelo le dió score alto dado que se deben buscar características que les den bajo score.

- **1878082** (pred=8733): Casó de fraude confirmado donde tuvieron que crear perfil transaccional en el exterior. Cliente es muy mayor de edad.
- **19073369** (pred=0.73): Llamada dicen estar en uruguay pero llaman al cliente y esta realmente en Bogota.
- **13250961** (pred=0.65): Según comentarios, parece ser un suplantador con los documentos reales del cliente que parece ser mayor.
- **39270054** (pred=0.53): Llamaron al cliente y el cliente esta en caucasia. Cliente enumerado. Le crearon perfil transaccional en muchos paises. Costa Rica, Brasil. Cliente con compras presenciales en Colombia.
- **42996405** (pred=0.54): Llaman y dice que la cliente está en Canada, sin embargo, encuentran un documento en oracle que la cliente tiene discapacidad cognitiva.
- **14956457**: Suplantación confirmada. Cliente confirmado en Buga.

Estudiar estos casos muy bien.

### Casos Sin Transacciones

In [ ]:
trns_totales = (ITC_Ips
 .groupby(['identificacion','fecha_hora_llamada'])
 .agg(numero_trn=('fechatrn','size')))
sin_trn = (df_clean
 .rename(columns={'FECHA':'fecha_hora_llamada',
                  'IDENTIFICACION':'identificacion',
                  'DESCCOD':'CodigoDesbloqueo'})
 .set_index(keys=['identificacion','fecha_hora_llamada'])
 .join(trns_totales, how='left')
 .query("numero_trn.isnull()")
)
sin_trn.shape

Tenemos 239 casos de llamadas sin ITC en el último mes

In [ ]:
cedulas_sin_ITC = sin_trn.reset_index().identificacion.tolist()
df_clean = df_clean[~df_clean.IDENTIFICACION.isin(cedulas_sin_ITC)]

In [ ]:
df_clean.shape

Nos quedan 8940 casos

### Suplantaciones Confirmadas

Es muy importante resaltar los casos con suplantación confirmada y que el modelo le dió score alto dado que se deben buscar características que les den bajo score.

- **1878082** (pred=8733): Casó de fraude confirmado donde tuvieron que crear perfil transaccional en el exterior. Cliente es muy mayor de edad.
- **19073369** (pred=0.73): Llamada dicen estar en uruguay pero llaman al cliente y esta realmente en Bogota.
- **13250961** (pred=0.65): Según comentarios, parece ser un suplantador con los documentos reales del cliente que parece ser mayor.
- **39270054** (pred=0.53): Llamaron al cliente y el cliente esta en caucasia. Cliente enumerado. Le crearon perfil transaccional en muchos paises. Costa Rica, Brasil. Cliente con compras presenciales en Colombia.
- **42996405** (pred=0.54): Llaman y dice que la cliente está en Canada, sin embargo, encuentran un documento en oracle que la cliente tiene discapacidad cognitiva.
- **14956457**: Suplantación confirmada. Cliente confirmado en Buga.

Estudiar estos casos muy bien.

### Multiples llamadas


- **3048552**: Tiene 3 llamadas detectadas como suplantaciones
- **94504254**: Tiene 3 llamadas con 2 detectadas como suplantaciones
- **1085309382**: Tiene 3 llamadas con 2 detectadas como suplantaciones

In [ ]:
plot_bars_percent_firstk(df=df_clean, colname='DESCCOD',
                         title='Frecuencia Tipos de Desbloqueos Después de Limpiar Datos')

# Modelo

## Creación Sabana de Datos

Sólo se toman los datos de **desbloqueo existoso** y **deteccion suplantacion** y se marcan aquellos desbloqueos exitosos que fueron resueltos en la primera llamada.

In [ ]:
df_clean.DESCCOD.unique().tolist()

In [ ]:
cod_to_int = {
    'Desbloqueo exitoso': 1,
    'Deteccion suplantacion ': 0,
    'Desbloqueo con soportes': 1
}
sub_df = (df_clean
 .assign(exitoso_primera_llamada = ((df_clean.groupby('IDENTIFICACION').FECHA.transform(lambda x: len(x))==1) &
                                     (df_clean.DESCCOD=='Desbloqueo exitoso')).astype(int) )
 .assign(exitoso_con_soportes = lambda x: (x.DESCCOD=='Desbloqueo con soportes').astype(int))
 .assign(label = lambda x: x.DESCCOD.map(cod_to_int))
 .query("label.isin([0,1])")
)
sub_df

### Transacciones Virtuales

In [ ]:
trns_totales = (ITC_Ips
 .groupby(['identificacion','fecha_hora_llamada'])
 .agg(numero_trn=('fechatrn','size')))

trns_totales_app_svp = (ITC_Ips[ITC_Ips.canal.str.strip().isin(['APP','SVP'])]
 .groupby(['identificacion','fecha_hora_llamada'])
 .agg(numero_trn_app_svp=('fechatrn','size')))

trns_totales_app_svp_localizadas = (ITC_Ips[ITC_Ips.canal.str.strip().isin(['APP','SVP']) & (ITC_Ips.country!='None')]
 .groupby(['identificacion','fecha_hora_llamada'])
 .agg(numero_trn_app_svp_localizadas=('fechatrn','size')))

trns_exterior_vs_colombia = (ITC_Ips[ITC_Ips.canal.str.strip().isin(['APP','SVP']) & (ITC_Ips.country!='None')]
 .assign(trn_colombia = lambda x: x.country=='CO')
 .assign(trns_exterior = lambda x: x.country!='CO')
 .groupby(['identificacion','fecha_hora_llamada'])
 .agg(numero_trn_col=('trn_colombia','sum'),
      numero_trn_ext=('trns_exterior','sum')))

dias_a_trn = (ITC_Ips
 .assign(canal = lambda x: x.canal.str.strip())
 .assign(fecha_hora_trn = lambda x: pd.to_datetime(x.fechatrn.astype(str) + 
                                                   ' ' + 
                                                   x.horatrn.astype(int).astype(str).str.pad(8,fillchar='0'),
                                                   format = '%Y%m%d %H%M%S%f') )
 .assign(dias_a_trn = lambda x: (x.fecha_hora_llamada - x.fecha_hora_trn) / pd.Timedelta(days=1))
 .query("dias_a_trn > 1.0")
)

ultima_trn = (dias_a_trn
 .sort_values(by=['identificacion','fecha_hora_llamada','fechatrn','horatrn'], ascending=False)
 .groupby(['identificacion','fecha_hora_llamada'])
 .first()
 .rename(columns={'dias_a_trn':'dias_a_ultima_trn',
                  'canal': 'canal_ultima_trn'})
 .loc[:,['dias_a_ultima_trn','canal_ultima_trn']]
)

ultima_trn_col = (dias_a_trn[dias_a_trn.country=='CO']
 .sort_values(by=['identificacion','fecha_hora_llamada','fechatrn','horatrn'], ascending=False)
 .groupby(['identificacion','fecha_hora_llamada'])
 .first()
 .rename(columns={'dias_a_trn':'dias_a_ultima_trn_col',
                  'canal': 'canal_ultima_trn_col'})
 .loc[:,['dias_a_ultima_trn_col','canal_ultima_trn_col']]
)

primera_trn_col = (dias_a_trn[dias_a_trn.country=='CO']
 .sort_values(by=['identificacion','fecha_hora_llamada','fechatrn','horatrn'], ascending=True)
 .groupby(['identificacion','fecha_hora_llamada'])
 .first()
 .rename(columns={'dias_a_trn':'dias_a_primera_trn_col',
                  'canal': 'canal_primera_trn_col'})
 .loc[:,['dias_a_primera_trn_col','canal_primera_trn_col']]
)

ultima_trn_ext = (dias_a_trn[(dias_a_trn.dias_a_trn > 1) & (~dias_a_trn.country.isin(['CO','None']))]
 .sort_values(by=['identificacion','fecha_hora_llamada','fechatrn','horatrn'], ascending=False)
 .groupby(['identificacion','fecha_hora_llamada'])
 .first()
 .rename(columns={'dias_a_trn':'dias_a_ultima_trn_ext',
                  'canal': 'canal_ultima_trn_ext'})
 .loc[:,['dias_a_ultima_trn_ext','canal_ultima_trn_ext']]
)

primera_trn_ext = (dias_a_trn[(dias_a_trn.dias_a_trn > 1) & (~dias_a_trn.country.isin(['CO','None']))]
 .sort_values(by=['identificacion','fecha_hora_llamada','fechatrn','horatrn'], ascending=True)
 .groupby(['identificacion','fecha_hora_llamada'])
 .first()
 .rename(columns={'dias_a_trn':'dias_a_primera_trn_ext'})
 .loc[:,['dias_a_primera_trn_ext']]
)

nro_countries = (ITC_Ips[ITC_Ips.country!='None']
 .groupby(['identificacion','fecha_hora_llamada'])
 .agg(nro_countries=('country','nunique')))

mean_lag_trns = (dias_a_trn[dias_a_trn.country!='None']
 .assign(trn_en_col = lambda x: 'mean_lag_trns_col_' + x.country.isin(['CO']).astype(int).astype(str))
 .groupby(['identificacion', 'fecha_hora_llamada', 'trn_en_col'])
 .agg(mean_lag_trns=('dias_a_trn','mean'))
 .reset_index()
 .pivot(index=['identificacion', 'fecha_hora_llamada'], columns='trn_en_col', values='mean_lag_trns')
 .fillna(-1)
 .assign(lag_diff_means = lambda x: x.mean_lag_trns_col_0 - x.mean_lag_trns_col_1)
)

### Transacciones Físicas

In [ ]:
def compras_fisicas(x):
    return (x.canal.str.strip().isin(['ABB','SAI']) | 
           (x.canal.str.strip().isin(['MII','RDB','PRD','MCI','PAX','PCM','PMR','PPM','PCR']) &
            ~x.cdgtrn.isin([226, 426, 526])))

compras_fisicas_raw = (dias_a_trn
 .assign(compra_fisica = lambda x: compras_fisicas(x))
 .assign(compra_fisica_nal = lambda x: (x.compra_fisica & (x.canal.str.strip().isin(['ABB','SAI']) | (x.campo5.str.strip()=='N')) ).astype(int))
 .assign(compra_fisica_int = lambda x: (x.compra_fisica & (x.campo5.str.strip()=='I')).astype(int))
 .assign(campo5 = lambda x: x.apply(lambda z: 'N' if z.compra_fisica_nal==1 else 'I',axis=1))
)

compras_fisicas = (compras_fisicas_raw[compras_fisicas_raw.compra_fisica]
 .sort_values(by=['identificacion','fecha_hora_llamada','fechatrn','horatrn'], ascending=False)
 .assign(campo5 = lambda x: x.campo5.str.strip())
 .groupby(['identificacion', 'fecha_hora_llamada', 'campo5'])
 .first()
 .reset_index()
 .rename(columns={'dias_a_trn_fisica':'dias_a_ultima_trn_ext'})
 .pivot(index=['identificacion', 'fecha_hora_llamada'] ,columns='campo5', values='dias_a_trn')
 .fillna(99)
 .reset_index()
 .assign(fisicas_int_mas_reciente = lambda x: (x.I <= x.N).astype(int) )
 .set_index(['identificacion', 'fecha_hora_llamada'])
 .loc[:,['I','N','fisicas_int_mas_reciente']]
)

conteo_compras_fisicas = (compras_fisicas_raw[compras_fisicas_raw.compra_fisica]
 .sort_values(by=['identificacion','fecha_hora_llamada','fechatrn','horatrn'], ascending=False)
 .groupby(['identificacion', 'fecha_hora_llamada'])
 .agg(compra_fisica_nal = ('compra_fisica_nal','sum'),
      compra_fisica_int = ('compra_fisica_int','sum'))
 .loc[:,['compra_fisica_nal','compra_fisica_int']]
)

### Códigos de Transferencias

In [ ]:
cdgtrn_seleccionados = (ITC_Ips[ITC_Ips.cdgtrn.isin([167,2])]
 .groupby(['identificacion','fecha_hora_llamada','cdgtrn'])
 .agg(count=('fechatrn','size'))
 .assign(count = lambda x: 1)
 .reset_index()
 .assign(cdgtrn = lambda x: x.cdgtrn.astype(int).astype(str))
 .pivot(index=['identificacion','fecha_hora_llamada'], columns='cdgtrn', values='count')
 .fillna(0)
 .astype(int)
)

### Combinación Features

In [ ]:
canales = ITC_Ips.canal.str.strip().value_counts().index.tolist()
canales_cod = dict(zip(canales, range(len(canales))))
canales_cod[-1] = -1
df_features_sub_org = (sub_df[['FECHA','IDENTIFICACION','exitoso_primera_llamada','exitoso_con_soportes','label']]
        .rename(columns={'FECHA':'fecha_hora_llamada',
                         'IDENTIFICACION':'identificacion'})
 .set_index(keys=['identificacion','fecha_hora_llamada'])
 .join(trns_totales)
 .join(trns_totales_app_svp)
 .join(trns_totales_app_svp_localizadas)
 .join(trns_exterior_vs_colombia)
 .join(nro_countries)
 .join(cdgtrn_seleccionados)
 .fillna(0)
 .astype(int)
 .assign(percent_ext = lambda x: x.apply(lambda z: 0 if z.numero_trn_app_svp_localizadas==0 
                                         else z.numero_trn_ext/z.numero_trn_app_svp_localizadas,axis=1))
 .assign(percent_col = lambda x: x.apply(lambda z: 0 if z.numero_trn_app_svp_localizadas==0 
                                         else z.numero_trn_col/z.numero_trn_app_svp_localizadas,axis=1))
 .assign(percent_app_svp = lambda x: x.apply(lambda z: 0 if z.numero_trn==0 
                                         else z.numero_trn_app_svp/z.numero_trn,axis=1))
 .join(ultima_trn)
 .join(ultima_trn_col)
 .join(primera_trn_col)
 .join(primera_trn_ext)
 .join(ultima_trn_ext)
 .fillna(99)
 .assign(canal_ultima_trn = lambda x: x.canal_ultima_trn.map(canales_cod))
 .assign(canal_ultima_trn_col = lambda x: x.canal_ultima_trn_col.map(canales_cod))
 .assign(canal_ultima_trn_ext = lambda x: x.canal_ultima_trn_ext.map(canales_cod))
 .join(mean_lag_trns)
 .join(compras_fisicas)
 .fillna(99)
 .assign(fisica_I = lambda x: (x.I!=99).astype(int))
 .assign(simultaneadad_fisica_N = lambda x: (x.N<x.dias_a_primera_trn_ext).astype(int))
 .join(conteo_compras_fisicas)
 .fillna(0)
#  .assign(dias_a_ultima_trn_col = lambda x: x[['dias_a_ultima_trn_col','N']].min(axis=1))
)
df_features_sub_org

In [ ]:
df_features_sub_org.groupby(['label','simultaneadad_fisica_N']).size()

Algo que realmente no me esperaba, pero casi el 22% de las llamadas con desbloqueo existoso cuentan con simultaneadad entre transacciones virtuales en el exterior y transacciones físicas en colombia.

Dado que esto es exactamente una señal de posible suplantación, entonces saquemos los registros de desbloqueos exitosos con simultaneidad que el modelo aprenda a darle scores bajos.

In [ ]:
# Se aplican los filtros para los datos donde se entrenará el modelo
df_features_sub = df_features_sub_org
# df_features_sub = (df_features_sub_org
#                    .query("(label==0) | ((label==1) & (simultaneadad_fisica_N==0) & (exitoso_primera_llamada==1) & (exitoso_con_soportes==0))")
#                    .copy())
df_features_sub.groupby(['label','simultaneadad_fisica_N']).size()
# df_features_sub.label.value_counts()

Con este cambio, solo quedamos con simultaneadades en suplantaciónes.

### Construcción X, y

In [ ]:
to_drop = [
    'identificacion',
    'fecha_hora_llamada',
    'label',
    'exitoso_primera_llamada',
    'exitoso_con_soportes',
    'numero_trn_app_svp_localizadas',
    'numero_trn_ext',
    'percent_col',
    'numero_trn_col',
    'numero_trn_app_svp',
    'canal_ultima_trn',
    'canal_ultima_trn_col',
    'canal_ultima_trn_ext',
    'dias_a_ultima_trn',
    'dias_a_ultima_trn_ext',
    'nro_countries',
    'mean_lag_trns_col_0',
    'mean_lag_trns_col_1',
    'lag_diff_means',
    'numero_trn',
    'N',
    'I',
    'fisicas_int_mas_reciente',
    'dias_a_primera_trn_col',
    'canal_primera_trn_col',
    'dias_a_primera_trn_ext',
    'percent_ext',
    '2',
    'compra_fisica_nal',
    'compra_fisica_int'
]
X = df_features_sub.reset_index().drop(columns=to_drop)
y = df_features_sub.label
X.shape,y.shape,y.value_counts()

## Modelado

### Train-Test Split

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state = 0) 
print("Shapes:")
print(train_X.shape, test_X.shape)
print("Test target distribution:")
print(test_Y.value_counts())

### Rendimiento CV

In [ ]:
get_oob_cv_balancedRF(train_X, train_Y, n_estimators = 300, n_repeats=5);

### Stepwise Check

In [ ]:
oob_ITCvars_filename = 'oob_ITCvars_v6.joblib'
if os.path.exists(oob_ITCvars_filename):
    obb_scores = joblib.load(oob_ITCvars_filename)
else:
    obb_scores = {col : get_oob_cv_balancedRF(train_X.drop(columns=col),
                                 train_Y, n_estimators = 300, 
                                 n_repeats=5) for col in X.columns}
    joblib.dump(obb_scores, oob_ITCvars_filename)

obb_scores   

### Rendimiento por Número de Arboles

In [ ]:
m = BalancedRandomForestClassifier(n_estimators=2000, random_state=0, oob_score=True).fit(train_X, train_Y)
preds_proba_train = m.oob_decision_function_[:,1]
preds_proba_test = m.predict_proba(test_X)[:,1]
# report_AUC(train_Y, preds_proba_train, data_set='training');
report_AUC(test_Y, preds_proba_test);

In [ ]:
plot_AUC_by_num_trees(extract_AUC_from_trees(m, test_X, test_Y))

In [ ]:
df_preds = pd.DataFrame({'y': test_Y}).assign(preds = preds_proba_test)
sns.displot(df_preds, x="preds", hue="y", kind="kde", fill=True)
plt.title("Test Prediction Scores by Class");
plt.xlim(0.0,1.0);

In [ ]:
plot_roc_curve_with_baseline(m, test_X, test_Y)

In [ ]:
fi = rf_feat_importance(m, train_X.columns).sort_values(by='imp', ascending=False)
plot_fi(fi)

In [ ]:
cluster_columns(X, figsize=(10,5), font_size=10)

Las variables parecen estár poco correlacionadas lo que es bastante bueno.

### Multiple Train-Test Repetitions

In [ ]:
def modelo_brf(random_state = 0):

    clf = BalancedRandomForestClassifier(n_estimators=500,random_state = random_state)
    
    return clf

mvm_filename = 'train_test_repetitions_var_metrica_v6.joblib'
if os.path.exists(mvm_filename):
    aucs, m_s, rocs = joblib.load(mvm_filename)
else:
    # Run test
    results = test_eval_repetitions(X, y, n_trials=100, model_func=modelo_brf)
    joblib.dump(results, mvm_filename)
    aucs, m_s, rocs = results

In [ ]:
print(f"Mean AUC: {round(np.mean(aucs),4)}, Std AUC: {round(np.std(aucs),4)}")
print(f"Percentiles (5,95): {tuple(np.quantile(aucs,[0.05, 0.95]).round(4))}")

In [ ]:
sns.displot(pd.DataFrame({'AUC': aucs}), x="AUC", kind="kde", fill=True)
plt.title('Distribution of AUC on Unseen Data');
plt.xlim(0.5,1.0);

In [ ]:
plot_rocs_from_multiple_runs(rocs, models_to_sample=0.13)

### LOO Predictions

#### Solo Suplantadores

El siguiente código nos permite calcular las predicciones LOO de solo los suplantadores para entender como les va sin tener que calcular las predicciones para todo el data set.

In [ ]:
suplantaciones = (df_features_sub[df_features_sub.label==0]
              .reset_index()
              .copy())
to_predict = list(suplantaciones.reset_index()[['identificacion','fecha_hora_llamada']].to_records(index=False))
to_predict
predictions_suplantacion = [None]*len(to_predict)
with tqdm(total=len(to_predict)) as pbar:
    for i,datos in enumerate(to_predict):
        cedula,fecha_hora = datos
        idx = np.where((df_features_sub.reset_index().identificacion==cedula) & 
                       (df_features_sub.reset_index().fecha_hora_llamada==fecha_hora))[0][0]
        X_train_, X_test_ = X[X.index!=idx], X[X.index==idx]
        y_train_, y_test_ = y[X.index!=idx], y[X.index==idx]
        m = (BalancedRandomForestClassifier(n_estimators=300, random_state=0)
             .fit(X_train_, y_train_))
        X_train_.shape, y_train_.shape, X_test_.shape, y_test_.shape
        predictions_suplantacion[i] = m.predict_proba(X_test_)[0,1]
        pbar.update(1)

suplantaciones['preds'] = predictions_suplantacion
suplantaciones.sort_values(by='preds', ascending=False).head(50)

In [ ]:
df_ = suplantaciones.assign(preds = lambda x: 1000*x.preds)
plots_bar_custom(df_, colname='binned', score_col='preds',
                 title='Número de Suplantadores por Banda de Score',show_text=True,
                 legend=None)

#### Todas las Predicciones

In [ ]:
def LOO_predictions(X, y, n_estimators=100):
    loo = LeaveOneOut()
    preds = [0]*len(X)
    with tqdm(total=len(X)) as pbar:
        for train_index, test_index in loo.split(X):
            X_train_, X_test_ = X.iloc[train_index], X.iloc[test_index]
            y_train_, y_test_ = y.iloc[train_index], y.iloc[test_index]
            m = (BalancedRandomForestClassifier(n_estimators=n_estimators, random_state=0)
                 .fit(X_train_, y_train_))
            preds[test_index[0]] = m.predict_proba(X_test_)[0,1]
            pbar.update(1)
    return preds

preds_filename = 'LOO_preds_v6.joblib'
if os.path.exists(preds_filename):
    preds = joblib.load(preds_filename)
else:
    preds = LOO_predictions(X,y,n_estimators=300)
    joblib.dump(preds, preds_filename)
len(preds)

In [ ]:
report_AUC(y, preds, data_set='LOO');

In [ ]:
df_sub_preds = df_features_sub.copy()
df_sub_preds['preds'] = preds
sns.displot(df_sub_preds, x="preds", hue="label", kind="hist", fill=True )
plt.title("Test Prediction Scores by Class");
plt.xlim(0.0,1.0);

In [ ]:
df_sub_preds[df_sub_preds.label==0].sort_values('preds', ascending=False).head(50)

In [ ]:
df_ = df_sub_preds[df_sub_preds.label==0].assign(preds = lambda x: 1000*x.preds)
supl_plot = plots_bar_custom(df_, colname='binned', score_col='preds',
                 title='Número de Suplantadores por Banda de Score',show_text=True,
                 legend=None)
df_ = df_sub_preds[df_sub_preds.label==1].assign(preds = lambda x: 1000*x.preds)
exit_plot = plots_bar_custom(df_, colname='binned', score_col='preds',
                 title='Número Desbloqueos Exitosos en Primera Llamada por Banda de Score',show_text=True,
                 legend=None)
supl_plot.properties(width=320) | exit_plot.properties(width=320)

Es tranquilizador que los los scores de los suplantadores se concentran en los scores bajos mientras que la de los desbloqueos exitosos al primer intento se concentran en los altos scores.

### Registros adicionales

In [ ]:
m_f = BalancedRandomForestClassifier(n_estimators=1000, random_state=0).fit(X, y)
datos_faltantes = (df_features_sub_org
                   .query("(label==1) & ((exitoso_primera_llamada==0) | (simultaneadad_fisica_N==1) | (exitoso_con_soportes==1) )")
                   .copy())
X_f = datos_faltantes.reset_index().drop(columns=to_drop)
y_f = datos_faltantes.label
preds_proba_f = m_f.predict_proba(X_f)[:,1]
datos_faltantes['preds'] = preds_proba_f

In [ ]:
df_ = datos_faltantes[datos_faltantes.simultaneadad_fisica_N==1].assign(preds = lambda x: 1000*x.preds)
simul = plots_bar_custom(df_, colname='binned', score_col='preds',
                 title='Exitosos con Simultaneidad por Banda de Score',show_text=True,
                 legend=None)
df_ = datos_faltantes[datos_faltantes.exitoso_primera_llamada==0].assign(preds = lambda x: 1000*x.preds)
varias = plots_bar_custom(df_, colname='binned', score_col='preds',
                 title='Exitosos en Varias Llamadas por Banda de Score',show_text=True,
                 legend=None)
df_ = datos_faltantes[datos_faltantes.exitoso_con_soportes==1].assign(preds = lambda x: 1000*x.preds)
soportes = plots_bar_custom(df_, colname='binned', score_col='preds',
                 title='Exitosos con Soportes por Banda de Score',show_text=True,
                 legend=None)
alt.vconcat(simul.properties(width=320) | varias.properties(width=320), soportes.properties(width=320))

Parece que el acercamiento funciona ya que el modelo le da un bajo score a los clientes con simultaneidad (scores debajo de 500). Por otro lado, los desbloqueos exitosos en varias llamadas y exitosos con soportes, tienen un comportamiento bastante parecido al desbloqueo de exitosos a la primera llamada con la adición de un pico en la banda de 200-300 que parece mostrar simultaneidad.

# Serializando Modelo

Se guarda el modelo en formato JOBLIB para luego ser usado en productivo.

In [ ]:
# joblib.dump(df_features_sub_org,'df_features_sub_org.joblib')

In [ ]:
col_l = df_features_sub_org.columns.tolist()
transformed_l = [ 'cdgtrn'+col if col in ['2','167'] else str(col).lower() for col in col_l]
col_transform = dict(zip(transformed_l,col_l))
cols_to_keep = list(df_features_sub_org.reset_index().drop(columns=to_drop).columns)

In [ ]:
model_pkg = {
    'model':m_f,
    'col_transform': col_transform,
    'cols_to_keep': cols_to_keep
}
joblib.dump(model_pkg,'model_pkg.joblib')

In [ ]:
# ITC_etiquetado[(ITC_etiquetado.canal.str.strip()=='SAI')].cdgtrn.value_counts()